In [60]:
import pandas as pd
import numpy as np
import re
import os

#Webscraping/NLP
import requests
from requests import get
from bs4 import BeautifulSoup
import time
import nltk
import requests
import unicodedata
from nltk.corpus import stopwords

#Visualizations
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#My imports
import prepare as p
import acquire as a

In [89]:
#Get the data
df = pd.read_pickle('data/michelin_df.pickle')
df.head()

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","Just a stone’s throw from Yonghe Temple, this ..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",This branch of the chain restaurant opened in ...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",A fixture on the city’s dining scene since 201...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",This multi-sensory experience at a secret loca...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","An air of refinement infuses this dining room,..."


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6780 entries, 0 to 6779
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     6780 non-null   object 
 1   address                  6780 non-null   object 
 2   location                 6780 non-null   object 
 3   price                    6726 non-null   object 
 4   cuisine                  6780 non-null   object 
 5   longitude                6780 non-null   float64
 6   latitude                 6780 non-null   float64
 7   phone_number             6649 non-null   float64
 8   url                      6780 non-null   object 
 9   website_url              5542 non-null   object 
 10  award                    6780 non-null   object 
 11  facilities_and_services  6732 non-null   object 
 12  data                     6780 non-null   object 
dtypes: float64(3), object(10)
memory usage: 688.7+ KB


In [ ]:
#docstring to make docstring and comments 

In [91]:
df.award.value_counts()

Bib Gourmand        3411
1 MICHELIN Star     2744
2 MICHELIN Stars     483
3 MICHELIN Stars     142
Name: award, dtype: int64

In [10]:
#TODO make visualization of distribution of Michelin star levels

In [92]:
df.price.value_counts()

€€      1337
€€€€    1261
€€€      854
$$       543
€        403
¥¥¥      359
¥¥       346
$$$$     288
$        228
¥        221
$$$      149
¥¥¥¥     139
฿฿       106
££        98
££££      89
฿         84
£££       68
₩         47
₩₩₩₩      28
฿฿฿฿      19
฿฿฿       15
£         12
₩₩        10
₺          8
₩₩₩        7
₺₺         3
₺₺₺        2
₺₺₺₺       2
Name: price, dtype: int64

In [52]:
#TODO drop phone number column?

In [93]:
#What is this column type
df.price.dtype

dtype('O')

In [94]:
df.price.isnull().values.any()

True

In [95]:
# Turn NaN values in price to nothing, so that it can be recast into the price_level column
df['price'] = df['price'].fillna('').astype('str')
df.price.value_counts()

€€      1337
€€€€    1261
€€€      854
$$       543
€        403
¥¥¥      359
¥¥       346
$$$$     288
$        228
¥        221
$$$      149
¥¥¥¥     139
฿฿       106
££        98
££££      89
฿         84
£££       68
          54
₩         47
₩₩₩₩      28
฿฿฿฿      19
฿฿฿       15
£         12
₩₩        10
₺          8
₩₩₩        7
₺₺         3
₺₺₺₺       2
₺₺₺        2
Name: price, dtype: int64

In [96]:
#Casting a new column, price level, using length of column
df['price_level'] = df['price'].apply(lambda x: len(x))

In [97]:
#Frequency of price levels in dataframe
df.price_level.value_counts()

2    2443
4    1826
3    1454
1    1003
0      54
Name: price_level, dtype: int64

In [101]:
#Setting mode of this column
mode= df.price_level.mode()[0]
mode

2

In [102]:
#Imputing 0s in price level with the mode for this column
df['price_level'] = df['price_level'].replace(0, mode)

In [103]:
df.price_level.value_counts()

2    2497
4    1826
3    1454
1    1003
Name: price_level, dtype: int64

In [104]:
#Separate City and country columns to create two new columns
df.location

0        Beijing, China Mainland
1        Beijing, China Mainland
2       Shanghai, China Mainland
3       Shanghai, China Mainland
4             San Francisco, USA
                  ...           
6775              Warsaw, Poland
6776              Cracow, Poland
6777              Cracow, Poland
6778              Warsaw, Poland
6779              Warsaw, Poland
Name: location, Length: 6780, dtype: object

In [105]:
#splitting location columns into two columns
df[['city', 'country']] = df['location'].str.split(',', 1, expand=True)

In [106]:
df.head()

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,data,price_level,city,country
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","Just a stone’s throw from Yonghe Temple, this ...",3,Beijing,China Mainland
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",This branch of the chain restaurant opened in ...,4,Beijing,China Mainland
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",A fixture on the city’s dining scene since 201...,4,Shanghai,China Mainland
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",This multi-sensory experience at a secret loca...,4,Shanghai,China Mainland
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","An air of refinement infuses this dining room,...",4,San Francisco,USA


In [107]:
#Dropping phone number column
df = df.drop(['phone_number'], axis=1)

In [108]:
#Dropping website url column
df = df.drop(['website_url'], axis=1)

In [109]:
#lower case all column values if column is object/string type
df = df.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)

In [110]:
df.head()

,name,address,location,price,cuisine,longitude,latitude,url,award,facilities_and_services,data,price_level,city,country
0,king's joy,"2 wudaoying hutong, beijing, china mainland","beijing, china mainland",¥¥¥,vegetarian,116.410004,39.946681,https://guide.michelin.com/en/beijing-municipa...,3 michelin stars,"air conditioning,american express credit card,...","just a stone’s throw from yonghe temple, this ...",3,beijing,china mainland
1,xin rong ji (xinyuan south road),"1f, east tower, genesis beijing, 8 xinyuan sou...","beijing, china mainland",¥¥¥¥,taizhou,116.450148,39.946380,https://guide.michelin.com/en/beijing-municipa...,3 michelin stars,"air conditioning,car park,china unionpay,count...",this branch of the chain restaurant opened in ...,4,beijing,china mainland
2,taian table,"101-102, building no. 1, garden office, no.161...","shanghai, china mainland",¥¥¥¥,innovative,121.474049,31.221807,https://guide.michelin.com/en/shanghai-municip...,3 michelin stars,"air conditioning,american express credit card,...",a fixture on the city’s dining scene since 201...,4,shanghai,china mainland
3,ultraviolet by paul pairet,'somewhere in shanghai' - meet at mr & mrs bun...,"shanghai, china mainland",¥¥¥¥,innovative,121.485090,31.240358,https://guide.michelin.com/en/shanghai-municip...,3 michelin stars,"air conditioning,american express credit card,...",this multi-sensory experience at a secret loca...,4,shanghai,china mainland
4,quince,"470 pacific ave., san francisco, 94133, usa","san francisco, usa",$$$$,"contemporary, californian",-122.403261,37.797505,https://guide.michelin.com/en/california/san-f...,3 michelin stars,"air conditioning,american express credit card,...","an air of refinement infuses this dining room,...",4,san francisco,usa


In [111]:
#Get clean and lemmatized text from reviews from prepare file
ret_df = p.process_nl(df.data)

In [112]:
ret_df.head()

,clean,lemmatized
0,stones throw yonghe temple serene restaurant r...,stone throw yonghe temple serene restaurant re...
1,branch chain restaurant opened 2019 features e...,branch chain restaurant opened 2019 feature el...
2,fixture citys dining scene since 2016 brainchi...,fixture city dining scene since 2016 brainchil...
3,multisensory experience secret location combin...,multisensory experience secret location combin...
4,air refinement infuses dining room thanks mura...,air refinement infuses dining room thanks mura...


In [113]:
df = pd.concat([df, ret_df], axis=1)

In [114]:
df.head()

,name,address,location,price,cuisine,longitude,latitude,url,award,facilities_and_services,data,price_level,city,country,clean,lemmatized
0,king's joy,"2 wudaoying hutong, beijing, china mainland","beijing, china mainland",¥¥¥,vegetarian,116.410004,39.946681,https://guide.michelin.com/en/beijing-municipa...,3 michelin stars,"air conditioning,american express credit card,...","just a stone’s throw from yonghe temple, this ...",3,beijing,china mainland,stones throw yonghe temple serene restaurant r...,stone throw yonghe temple serene restaurant re...
1,xin rong ji (xinyuan south road),"1f, east tower, genesis beijing, 8 xinyuan sou...","beijing, china mainland",¥¥¥¥,taizhou,116.450148,39.946380,https://guide.michelin.com/en/beijing-municipa...,3 michelin stars,"air conditioning,car park,china unionpay,count...",this branch of the chain restaurant opened in ...,4,beijing,china mainland,branch chain restaurant opened 2019 features e...,branch chain restaurant opened 2019 feature el...
2,taian table,"101-102, building no. 1, garden office, no.161...","shanghai, china mainland",¥¥¥¥,innovative,121.474049,31.221807,https://guide.michelin.com/en/shanghai-municip...,3 michelin stars,"air conditioning,american express credit card,...",a fixture on the city’s dining scene since 201...,4,shanghai,china mainland,fixture citys dining scene since 2016 brainchi...,fixture city dining scene since 2016 brainchil...
3,ultraviolet by paul pairet,'somewhere in shanghai' - meet at mr & mrs bun...,"shanghai, china mainland",¥¥¥¥,innovative,121.485090,31.240358,https://guide.michelin.com/en/shanghai-municip...,3 michelin stars,"air conditioning,american express credit card,...",this multi-sensory experience at a secret loca...,4,shanghai,china mainland,multisensory experience secret location combin...,multisensory experience secret location combin...
4,quince,"470 pacific ave., san francisco, 94133, usa","san francisco, usa",$$$$,"contemporary, californian",-122.403261,37.797505,https://guide.michelin.com/en/california/san-f...,3 michelin stars,"air conditioning,american express credit card,...","an air of refinement infuses this dining room,...",4,san francisco,usa,air refinement infuses dining room thanks mura...,air refinement infuses dining room thanks mura...


In [115]:
df.award.dtype

dtype('O')

In [41]:
#TODO Create clean dtypes function

In [ ]:
#drop other two columns

In [124]:
df[df.country.isnull()]

,name,address,location,price,cuisine,longitude,latitude,url,award,facilities_and_services,data,price_level,city,country,clean,lemmatized
69,les amis,"shaw centre, #01-16, 1 scotts road, 228208, si...",singapore,$$$$,french,103.832040,1.306260,https://guide.michelin.com/en/singapore-region...,3 michelin stars,"air conditioning,american express credit card,...",les amis is as good-looking today as it was wh...,4,singapore,None,les amis goodlooking today opened 1994 singula...,le amis goodlooking today opened 1994 singular...
70,odette,"national gallery, #01-04, 1 st. andrew's road ...",singapore,$$$$,french contemporary,103.851600,1.289646,https://guide.michelin.com/en/singapore-region...,3 michelin stars,"air conditioning,american express credit card,...",where better to find a priceless jewel of the ...,4,singapore,None,better find priceless jewel singapore dining s...,better find priceless jewel singapore dining s...
71,zén,"41 bukit pasoh road, 089855, singapore",singapore,$$$$,european contemporary,103.840420,1.279627,https://guide.michelin.com/en/singapore-region...,3 michelin stars,"air conditioning,american express credit card,...",the celebrity chef of frantzén fame opened his...,4,singapore,None,celebrity chef frantzen fame opened third asia...,celebrity chef frantzen fame opened third asia...
132,jade dragon,"2f, the shops at the boulevard, city of dreams...",macau,$$$,cantonese,113.567150,22.149137,https://guide.michelin.com/en/macau-region/mac...,3 michelin stars,"air conditioning,american express credit card,...",this lavish room is a destination in itself – ...,3,macau,None,lavish room destination chinese art ebony crys...,lavish room destination chinese art ebony crys...
133,robuchon au dôme,"43f, grand lisboa hotel, avenida de lisboa, macau",macau,$$$$,french contemporary,113.543950,22.189949,https://guide.michelin.com/en/macau-region/mac...,3 michelin stars,"air conditioning,american express credit card,...",its lofty position atop the grand lisboa hotel...,4,macau,None,lofty position atop grand lisboa hotel lets di...,lofty position atop grand lisboa hotel let din...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6728,nishiki,"shop 103, 1f, regal kowloon hotel, 71 mody roa...",hong kong,$$,japanese,114.178034,22.299056,https://guide.michelin.com/en/hong-kong-region...,bib gourmand,"air conditioning,american express credit card,...","despite opening over 20 years ago, the bustlin...",2,hong kong,None,despite opening 20 years ago bustling atmosphe...,despite opening 20 year ago bustling atmospher...
6729,café hunan (western district),"420-424 queen's road west, hong kong",hong kong,$$,hunanese,114.138742,22.286426,https://guide.michelin.com/en/hong-kong-region...,bib gourmand,"air conditioning,american express credit card,...",the hunanese chef honed his skills at his moth...,2,hong kong,None,hunanese chef honed skills mothers restaurant ...,hunanese chef honed skill mother restaurant pa...
6730,shugetsu ramen (central),"5 gough street, hong kong",hong kong,$$,ramen,114.152936,22.284133,https://guide.michelin.com/en/hong-kong-region...,bib gourmand,"air conditioning,counter dining,credit card / ...","the queues form early for the ramen here, with...",2,hong kong,None,queues form early ramen many customers coming ...,queue form early ramen many customer coming ts...
6731,tasty (central),"shop 3016-3018, 3f, ifc mall, 1 harbour view s...",hong kong,$$,noodles and congee,114.158160,22.285889,https://guide.michelin.com/en/hong-kong-region...,bib gourmand,"air conditioning,american express credit card,...",be prepared to queue and share a table because...,2,hong kong,None,prepared queue share table 1000 customers day ...,prepared queue share table 1000 customer day m...


In [127]:
#Turn city/states into city states, impute city into country column
df['country'] = np.where(pd.isna(df['country']), df['city'], df['country'])

In [132]:
df[df.country == 'hong kong']

,name,address,location,price,cuisine,longitude,latitude,url,award,facilities_and_services,data,price_level,city,country,clean,lemmatized
135,8 1/2 otto e mezzo - bombana,"shop 202, 2f, alexandra house, 18 chater road,...",hong kong,$$$,italian,114.158670,22.281464,https://guide.michelin.com/en/hong-kong-region...,3 michelin stars,"air conditioning,american express credit card,...",owner-chef umberto bombana’s restaurant oozes ...,3,hong kong,hong kong,ownerchef umberto bombanas restaurant oozes it...,ownerchef umberto bombanas restaurant ooze ita...
136,t'ang court,"1-2f, the langham hotel, 8 peking road, hong kong",hong kong,$$$,cantonese,114.169800,22.296572,https://guide.michelin.com/en/hong-kong-region...,3 michelin stars,"air conditioning,american express credit card,...",it’s easy to see why this restaurant remains s...,3,hong kong,hong kong,easy see restaurant remains popular opening do...,easy see restaurant remains popular opening do...
137,sushi shikon,"7f, the landmark mandarin oriental hotel, 15 q...",hong kong,$$$$,sushi,114.157416,22.280857,https://guide.michelin.com/en/hong-kong-region...,3 michelin stars,"air conditioning,american express credit card,...",the famous sushi-ya moved to this prestigious ...,4,hong kong,hong kong,famous sushiya moved prestigious address 2019 ...,famous sushiya moved prestigious address 2019 ...
138,caprice,"6f, four seasons hotel, 8 finance street, hong...",hong kong,$$$$,french contemporary,114.159050,22.285715,https://guide.michelin.com/en/hong-kong-region...,3 michelin stars,"air conditioning,american express credit card,...",one of the most glamorous and elegant restaura...,4,hong kong,hong kong,one glamorous elegant restaurants hong kong al...,one glamorous elegant restaurant hong kong als...
139,forum,"1f, sino plaza, 255-257 gloucester road, hong ...",hong kong,$$$$,cantonese,114.182199,22.281440,https://guide.michelin.com/en/hong-kong-region...,3 michelin stars,"air conditioning,american express credit card,...","the iconic dish here, ah yat braised abalone, ...",4,hong kong,hong kong,iconic dish ah yat braised abalone become famo...,iconic dish ah yat braised abalone become famo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6728,nishiki,"shop 103, 1f, regal kowloon hotel, 71 mody roa...",hong kong,$$,japanese,114.178034,22.299056,https://guide.michelin.com/en/hong-kong-region...,bib gourmand,"air conditioning,american express credit card,...","despite opening over 20 years ago, the bustlin...",2,hong kong,hong kong,despite opening 20 years ago bustling atmosphe...,despite opening 20 year ago bustling atmospher...
6729,café hunan (western district),"420-424 queen's road west, hong kong",hong kong,$$,hunanese,114.138742,22.286426,https://guide.michelin.com/en/hong-kong-region...,bib gourmand,"air conditioning,american express credit card,...",the hunanese chef honed his skills at his moth...,2,hong kong,hong kong,hunanese chef honed skills mothers restaurant ...,hunanese chef honed skill mother restaurant pa...
6730,shugetsu ramen (central),"5 gough street, hong kong",hong kong,$$,ramen,114.152936,22.284133,https://guide.michelin.com/en/hong-kong-region...,bib gourmand,"air conditioning,counter dining,credit card / ...","the queues form early for the ramen here, with...",2,hong kong,hong kong,queues form early ramen many customers coming ...,queue form early ramen many customer coming ts...
6731,tasty (central),"shop 3016-3018, 3f, ifc mall, 1 harbour view s...",hong kong,$$,noodles and congee,114.158160,22.285889,https://guide.michelin.com/en/hong-kong-region...,bib gourmand,"air conditioning,american express credit card,...",be prepared to queue and share a table because...,2,hong kong,hong kong,prepared queue share table 1000 customers day ...,prepared queue share table 1000 customer day m...


In [136]:
#For testing this function, getting clean data
df = pd.read_pickle('data/michelin_df.pickle')
df.head()

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","Just a stone’s throw from Yonghe Temple, this ..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",This branch of the chain restaurant opened in ...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",A fixture on the city’s dining scene since 201...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",This multi-sensory experience at a secret loca...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","An air of refinement infuses this dining room,..."


In [137]:
def create_features(df):
    '''
    This function takes in the dataframe, drops unnecessary columns,
    and creates new columns/features for exploration and potential
    classification purposes. It returns the dataframe with applications
    '''
    # Dropping unnecessary columns
    df = df.drop(['phone_number', 'website_url'], axis=1)
    
    #lower case all column values if column is object/string type
    df = df.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)
    
    # Turn NaN values in price to 'nothing', so that it can be recast into the price_level column
    df['price'] = df['price'].fillna('').astype('str')
    # Casting a new column, price level, using length of column
    df['price_level'] = df['price'].apply(lambda x: len(x))
    # impute price level "0" with the mode for this column
    df['price_level'] = df['price_level'].replace(0, mode)
    
    # splitting location columns into two columns
    df[['city', 'country']] = df['location'].str.split(',', 1, expand=True)
    # Turn city/states into city states, impute city into country column (i.e Hong Kong, Hong Kong)
    df['country'] = np.where(pd.isna(df['country']), df['city'], df['country'])

    return df
    
    

In [138]:
df = create_features(df)

In [139]:
df.head()

,name,address,location,price,cuisine,longitude,latitude,url,award,facilities_and_services,data,price_level,city,country
0,king's joy,"2 wudaoying hutong, beijing, china mainland","beijing, china mainland",¥¥¥,vegetarian,116.410004,39.946681,https://guide.michelin.com/en/beijing-municipa...,3 michelin stars,"air conditioning,american express credit card,...","just a stone’s throw from yonghe temple, this ...",3,beijing,china mainland
1,xin rong ji (xinyuan south road),"1f, east tower, genesis beijing, 8 xinyuan sou...","beijing, china mainland",¥¥¥¥,taizhou,116.450148,39.946380,https://guide.michelin.com/en/beijing-municipa...,3 michelin stars,"air conditioning,car park,china unionpay,count...",this branch of the chain restaurant opened in ...,4,beijing,china mainland
2,taian table,"101-102, building no. 1, garden office, no.161...","shanghai, china mainland",¥¥¥¥,innovative,121.474049,31.221807,https://guide.michelin.com/en/shanghai-municip...,3 michelin stars,"air conditioning,american express credit card,...",a fixture on the city’s dining scene since 201...,4,shanghai,china mainland
3,ultraviolet by paul pairet,'somewhere in shanghai' - meet at mr & mrs bun...,"shanghai, china mainland",¥¥¥¥,innovative,121.485090,31.240358,https://guide.michelin.com/en/shanghai-municip...,3 michelin stars,"air conditioning,american express credit card,...",this multi-sensory experience at a secret loca...,4,shanghai,china mainland
4,quince,"470 pacific ave., san francisco, 94133, usa","san francisco, usa",$$$$,"contemporary, californian",-122.403261,37.797505,https://guide.michelin.com/en/california/san-f...,3 michelin stars,"air conditioning,american express credit card,...","an air of refinement infuses this dining room,...",4,san francisco,usa
